In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from selenium import webdriver
import time
import numpy as np
import time
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
def getNumPages(self,password =''):
    """
    Calculates the number of pages in this PDF file.

    :return: number of pages
    :rtype: int
    :raises PdfReadError: if file is encrypted and restrictions prevent
        this action.
    """

    # Flattened pages will not work on an Encrypted PDF;
    # the PDF file's page count is used in this case. Otherwise,
    # the original method (flattened page count) is used.
    if self.isEncrypted:
        try:
            self._override_encryption = True
            self.decrypt(password)
            return self.trailer["/Root"]["/Pages"]["/Count"]
        except:
            raise utils.PdfReadError("File has not been decrypted")
        finally:
            self._override_encryption = False
    else:
        if self.flattenedPages == None:
            self._flatten()
        return len(self.flattenedPages)

numPages = property(lambda self: self.getNumPages(), None, None)

# Webscraping and pdf downloading

I am specifically focusing on recent General Obligation Bonds which fall into the 'other' category on the [EMMA](https://emma.msrb.org/Search/Search.aspx) website. I ran this twice as there were too many issues to load onto the EMMA site in one iteration. Below the dates can be adjusted to replicated the dataset. I ended up getting documents for bonds between 06/01/2020 - 11/4/2020 for the final project.

In [6]:
# indicate where in my local machine the geckodriver is for using firefox.
driver = webdriver.Firefox(executable_path=r'C:/Users/atag3/Desktop/NYC DSA Python EDA/Untitled Folder/geckodriver.exe')

# we have to click through the first message accepting a disclaimer.
driver.get('https://emma.msrb.org/Search/Search.aspx')
butt = driver.find_elements_by_xpath('//*[@id="ctl00_mainContentArea_disclaimerContent_yesButton"]')
butt[0].click()

In [7]:
from selenium.webdriver.support.ui import Select

select = Select(driver.find_element_by_id('purposeDropdownList'))

# select purpose of muni bond as other
select.select_by_visible_text('Other')

select = Select(driver.find_element_by_id('securedByDropdownList'))

# select repayment type as General Obligation Bond
select.select_by_visible_text('General Obligation')

driver.find_element_by_xpath('//*[@id="datedDateFrom"]').send_keys('06/01/2020')
driver.find_element_by_xpath('//*[@id="datedDateTo"]').send_keys('08/30/2020')

driver.find_element_by_xpath('//*[@id="runSearchButton"]').click()
time.sleep(4)

#change view to be by issuers
driver.find_element_by_xpath('//*[@id="issuesTab"]').click()

# 
select = Select(driver.find_element_by_name('lvIssues_length'))

# select purpose of muni bond as other
select.select_by_visible_text('100')

driver.find_element_by_xpath('/html/body/form/div[10]/div/div/div[2]/button').click()

ElementNotInteractableException: Message: Element <a id="issuesTab"> could not be scrolled into view


In [ ]:
# now that we have the page the way we want it to look, we can look through all the pages for bond names issuer names and links
max_n = int(driver.find_element_by_id('lvIssues_info').text.split(' ')[5].replace(',',''))

#Create an empty dataframe to save information to.
doc_table = pd.DataFrame(columns=['Issuer Name', 'Description', 'State','Dated Date','link'])

# num of pages to go through.
its = int(np.ceil(max_n/100))

for i in range(0,its):
    html = driver.page_source
    table = pd.read_html(html)[4]
    soup = BeautifulSoup(html, 'html.parser')
    
    # look for the link to bond issue page.
    table['link'] = [item.get('href') for item in soup.find_all("a", href=re.compile("IssueView"))]
    
    doc_table = doc_table.append(table,ignore_index=True)
    time.sleep(5)
    
    #click to next page and run through the loop again.
    driver.find_element_by_xpath('/html/body/form/div[9]/div[3]/div[2]/div[2]/div[6]/div[3]/div/div/div[5]/a[3]').click()

In [ ]:
#crteate a bond id using the final section of the link. Note this is not a cusip.
doc_table['ID']= [link.split('/')[-1] for link in doc_table['link']]

In [ ]:
# run through our list of links and downlaod the pdfs.
for i in range(6,len(doc_table)):
    try:
        link= doc_table['link'][i]
        name= doc_table['ID'][i]
        url = link.replace('..','https://emma.msrb.org')
        driver.get(url)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="ui-id-4"]').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # we are extracting the link to the official pdf from the page and downloading the file. In my case I am saving it to
        # a specific folder I have created on a backup HDD drive since the pdf files can take up a lot of space.
        
        pdf_link = [item.get('href') for item in soup.find_all("a", href=re.compile(".pdf"))][0]
        urllib.request.urlretrieve('https://emma.msrb.org'+pdf_link, 'E:/Users/atag3/Documents/Project_Data/Pull2/'+name+'.pdf')
        time.sleep(5)
    except Exception:
        #print name if error occurs.
        print(name)
    

In [ ]:
import glob
from pikepdf import Pdf

# get list of pdfs ion the previously used folder
txtfiles = []
for file in glob.glob("E:/Users/atag3/Documents/Project_Data/Pull2/*.pdf"):
    txtfiles.append(file.replace('\\','/'))
    
#decrypt them and save them to a different folder using the same ID from the doc_table dataframe.
for file in txtfiles:
    try:
        new_pdf = Pdf.new()
        with Pdf.open(file) as pdf:
            pdf.save('E:/Users/atag3/Documents/Project_Data/Pull2/decrypted/'+file.split('/')[-1])
    except Exception:
        print(file)

In [ ]:
# Create a new list of the decrypted pdfs.
txtfiles = []
for file in glob.glob("E:/Users/atag3/Documents/Project_Data/Pull2/decrypted/*.pdf"):
    txtfiles.append(file.replace('\\','/'))

In [ ]:
# Create a dictionary and read in all of the pdfs we had downloaded.
go_text = {}
for file in txtfiles:
    try:
        #open allows you to read the file.
        pdfFileObj = open(file,'rb')
        #The pdfReader variable is a readable object that will be parsed.
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

        #Discerning the number of pages will allow us to parse through all the pages.
        num_pages = pdfReader.numPages
        count = 0
        text = ""
        #The while loop will read each page.
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()
        #This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
        if text != "":
            text = text
        #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text.
        else:
            #this might be able to help with document scans but I was running into issues and it only helped with a few documents.
            #text = textract.process(fileurl, method='tesseract', language='eng')
            continue
        #Now we have a text variable that contains all the text derived from 

        go_text[file] = {'doc':text}
    except Exception:
        print(file)

At this point I pickled the two versions and recombined them after the fact. I also removed documents with a word count below 10,000 which seemed to be mostly amendment documents to the official issuing document or other reports. 

In [9]:
#import pickle
#big_df2 = pd.DataFrame.from_dict(go_text, orient='index').dropna(how='all',axis=1)
#big_df2.to_pickle("./doc_text_2.pkl")
muni_df = pd.read_pickle('muni_df.pkl')
muni_df['len'] = muni_df['doc'].str.len()
muni_df.head()

,Issuer Name,Description,State,Dated Date,link,ID,doc,len
2,"ADDISON PARK DISTRICT, DUPAGE COUNTY, ILLINOIS","GENERAL OBLIGATION LIMITED TAX PARK BONDS, SER...",IL,09/30/2020,../IssueView/Details/P1404329,P1404329,NEW ISSUES \nS&P Global Ratings: \nBOOK-EN...,300745.0
3,"ADDISON PARK DISTRICT, DUPAGE COUNTY, ILLINOIS",GENERAL OBLIGATION REFUNDING PARK BONDS (ALTER...,IL,09/30/2020,../IssueView/Details/P1404328,P1404328,NEW ISSUES \nS&P Global Ratings: \nBOOK-EN...,300745.0
4,"ADIRONDACK CENTRAL SCHOOL DISTRICT, NEW YORK","GENERAL OBLIGATIONS BOND ANTICIPATION NOTES, 2020",NY,09/16/2020,../IssueView/Details/P2403616,P2403616,\nOFFICIAL STATEMENT\n \nNEW ISSUE\n \nBOND A...,294184.0
5,ALABAMA TOWN NEW YORK,"BOND ANTICIPATION NOTES, 2020 SERIES A",NY,10/01/2020,../IssueView/Details/P1404548,P1404548,\n \nFINAL \nOFFICIAL STATEMENT DATED \nSEPTE...,150332.0
7,ALFRED-ALMOND CENTRAL SCHOOL DISTRICT ALLEGANY...,"BOND ANTICIPATION NOTES, 2020",NY,09/16/2020,../IssueView/Details/P1404126,P1404126,\n \nFINAL \nOFFICIAL STATEMENT DATED\n \nSEP...,159201.0


# Preprocessing Text

Note that because of the length of some of the documents this can take over an hour or two to finish

In [10]:
# a table of states and cities to try and use as stop words to remove geographic words from documents.
cities = pd.read_csv('https://raw.githubusercontent.com/grammakov/USA-cities-and-states/master/us_cities_states_counties.csv', sep="|")

In [11]:
states= cities['State full'].unique().tolist()
states = [x for x in states if pd.notnull(x)]
states = [x.lower() for x in states]
states= [words for segments in states for words in segments.split()]

In [14]:
#A bunch of specific stop words I added plus state names.
specific_stops = ['texas',
                 'city',
                 'town',
                 'county',
                  'audit',
                 'bidder',
                 'township',
                  'counties',
                  'tax',
                  'state',
                  'bond',
                  'bonds',
                  'fort',
                  'bend',
                  'cities',
                  'towns'
                  'taxable',
                  'disbursments',
                  'bond',
                 'billion', 
                  'stateTMs', 
                  'thousands',
                  'proposition',
                  'village',  
                  'obetz', 
                  'villagetms',
                  'cityTMs', 
                  'countyTMs', 
                  'commonwealth',
                  'huntsville',
                  'inflows', 
                  'basic', 
                  'outflows', 
                  'fiduciary', 
                  'wide', 
                  'actuarial',
                  'return', 
                  'component',
                  'depreciation',
                  'oxford', 
                  'birmingham', 
                  'madison', 
                  'cityTMs',
                  'districtTMs', 
                  'harris', 
                  'houston',
                  'williamson', 
                  'rutherford',
                  'municipalityTMs',
                  'cont',
                  'borough',
                  'lexington', 
                  'fayette', 
                  'des', 
                  'san',
                  'antonio',
                  'francisco',
                  'bay',
                  'cpa',
                  'travis',
                  'austin'
                  'miami',
                  'opeb',
                   'msrb',
                  'quincy',
                  'ehlers', 
                  'wrs', 
                  'brady',
                  'quarles',
                  'evers', 
                  'btsc', 
                  'subdivisionTMs', 
                  'underwriter',
                  'registrar', 
                  'receipts', 
                  'dtcTMs', 
                  'cusip', 
                  'hereof',
                  'jefferson',
                  'townTMs',
                  'bam', 
                  'ict', 
                  'rict',
                  'trict',
                  'distr',
                  'distric', 
                  'istrict',
                  'comptroller',
                  'milwaukee',
                  'illion'
                  'chicago',
                  'agm',
                  'municipality',
                  'municipalityTMs',
                  'promissory',
                  'left',
                  'intentionally',
                  'blank',
                  'moines']+states 

In [15]:
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
from sklearn.feature_extraction import text 


def preprocess(docs):
    return_list = []
    for content in docs:
        content = content.lower()
        content = content.replace('\n','')

        content = "".join([char for char in content if char not in string.punctuation])

        words = word_tokenize(content)

        stop_words = text.ENGLISH_STOP_WORDS.union(specific_stops)
        words = [word for word in words if word not in stop_words]

        porter = PorterStemmer()
        words= [porter.stem(word) for word in words]
        return_list.append(words)
    return return_list

muni_df = muni_df.reset_index(drop=True)

clean_text = preprocess(muni_df['doc'])



#cat version for passing to vectorizer
clean_text_cat = list(map(lambda x: ' '.join(x),clean_text))

In [16]:
muni_df['clean_text'] = clean_text_cat

In [21]:
muni_df.to_pickle("./muni_clean.pkl")